# ViSoND Notebook (One-step upload + render)
Upload your CSV or NPY file (columns fixed as):
Rows: Each discrete event
Columns:
0 = Track: which instrument track is assigned
1 = Channel: not used
2 = Note: identity of event (e.g., neuron index, sniff rate, speed)
3 = Velocity: intensity of event (if event is binary (e.g., spikes), set to 127)  
4 = Note-on: event time start in seconds 
5 = Note-off event emd time in seconds 

In [2]:
# ViSoND one-shot UI — upload + options + STRICT unique scale-ladder mapping + render
# If unique note IDs exceed scale capacity, raises: ValueError("Too many notes ...")

import io, numpy as np, pandas as pd, mido
import ipywidgets as widgets
from mido import MetaMessage
from IPython.display import display, HTML, FileLink

# ---------------- UI ----------------
upload = widgets.FileUpload(accept='.csv,.npy', multiple=False)
bpm_slider = widgets.IntSlider(value=120, min=20, max=960, description='BPM')
tpb_slider = widgets.IntSlider(value=5000, min=16, max=10000, description='Ticks/QN')
root_dd = widgets.Dropdown(options=['C','C#','D','D#','E','F','F#','G','G#','A','A#','B'],
                           value='C', description='Root')
scale_dd = widgets.Dropdown(options=[], value=None, description='Scale', layout=widgets.Layout(width='360px'))
render_btn = widgets.Button(description='Render MIDI', button_style='success')
log = widgets.Output()

display(widgets.VBox([
    widgets.HTML("<b>Upload .csv or .npy (cols 0..5: track,channel,note,velocity,start_s,end_s)</b>"),
    upload,
    widgets.HBox([bpm_slider, tpb_slider]),
    widgets.HBox([root_dd, scale_dd]),
    render_btn,
    widgets.HTML("<b>Log</b>"),
    log
]))

# ------------- Helpers & Scales -------------
def _get_uploaded_name_and_bytes(upl):
    v = upl.value
    if not v: return None, None
    if isinstance(v, dict):  # ipywidgets v7
        name, info = next(iter(v.items()))
        return name, info['content']
    else:  # ipywidgets v8 -> tuple of UploadedFile
        f = v[0]
        name = getattr(f, 'name', None) or f.get('name')
        content = getattr(f, 'content', None) or f.get('content')
        return name, content

NOTE_TO_SEMITONE = {'C':0,'C#':1,'D':2,'D#':3,'E':4,'F':5,'F#':6,'G':7,'G#':8,'A':9,'A#':10,'B':11}

# Extended scale set (12-TET approximations where needed)
SCALE_INTERVALS = {
    # Chromatic
    "Chromatic (12-tone)": list(range(12)),

    # 7-note modes
    "Ionian (Major)"            : [0, 2, 4, 5, 7, 9, 11],
    "Dorian"                    : [0, 2, 3, 5, 7, 9, 10],
    "Phrygian"                  : [0, 1, 3, 5, 7, 8, 10],
    "Lydian"                    : [0, 2, 4, 6, 7, 9, 11],
    "Mixolydian"                : [0, 2, 4, 5, 7, 9, 10],
    "Aeolian (Natural Minor)"   : [0, 2, 3, 5, 7, 8, 10],
    "Locrian"                   : [0, 1, 3, 5, 6, 8, 10],

    # Pentatonics (incl. Japanese variants)
    "Pentatonic Major"                 : [0, 2, 4, 7, 9],
    "Pentatonic Minor"                 : [0, 3, 5, 7, 10],
    "Pentatonic Egyptian"              : [0, 2, 5, 7, 10],
    "Pentatonic Yo (Japanese)"         : [0, 2, 5, 7, 9],
    "Pentatonic In Sen (Japanese)"     : [0, 1, 5, 7, 10],
    "Pentatonic Hirajoshi (Japanese)"  : [0, 2, 3, 7, 8],
    "Pentatonic Iwato (Japanese)"      : [0, 1, 5, 6, 10],
    "Pentatonic Dorian"                : [0, 2, 3, 7, 9],

    # Blues (hexatonics)
    "Blues Minor (hexatonic)" : [0, 3, 5, 6, 7, 10],
    "Blues Major (hexatonic)" : [0, 2, 3, 4, 7, 9],

    # Chinese pentatonic modes (五声) — 12-TET approximations
    "Chinese Gong (宫)" : [0, 2, 4, 7, 9],
    "Chinese Shang (商)": [0, 2, 5, 7, 10],
    "Chinese Jiao (角)" : [0, 3, 5, 8, 10],
    "Chinese Zhi (徵)"  : [0, 2, 5, 7, 9],
    "Chinese Yu (羽)"   : [0, 3, 5, 7, 10],

    # Ethiopian qenet modes — 12-TET approximations
    "Ethiopian Tezeta Major": [0, 2, 4, 7, 9],
    "Ethiopian Tezeta Minor": [0, 3, 5, 7, 10],
    "Ethiopian Bati Major"  : [0, 2, 4, 7, 9],
    "Ethiopian Bati Minor"  : [0, 2, 3, 7, 9],
    "Ethiopian Ambassel"    : [0, 3, 5, 7, 10],
    "Ethiopian Anchihoye"   : [0, 2, 3, 7, 9],
}
scale_dd.options = list(SCALE_INTERVALS.keys())
scale_dd.value = "Chromatic (12-tone)"

def build_allowed_pitches(root, scale, lo=0, hi=127):
    root_pc = NOTE_TO_SEMITONE[root]
    pattern = SCALE_INTERVALS[scale]
    return np.array([n for n in range(lo, hi+1) if (n - root_pc) % 12 in pattern], dtype=int)

def map_unique_to_scale_ladder_strict(values, root, scale, base_note=None):
    """
    Order-preserving, 1:1 mapping into a scale ladder.
    Raises a ValueError("Too many notes ...") if there are more unique values than capacity (0..127).
    - Works with numbers, strings, None/NaN; preserves first-appearance order for non-numeric.
    """
    vals = pd.Series(values)
    uniq = vals.dropna().drop_duplicates().tolist()
    nuniq = len(uniq)

    allowed = build_allowed_pitches(root, scale, lo=0, hi=127)
    if allowed.size == 0:
        raise ValueError(f"Scale '{scale}' with root '{root}' has no allowed pitches in 0–127.")

    # Choose ladder start (base_note reserved for future UI; default = start at first allowed)
    start_idx = 0 if base_note is None else int(np.searchsorted(allowed, np.clip(int(base_note), 0, 127)))
    capacity = max(0, allowed.size - start_idx)
    if capacity <= 0:
        start_idx = 0
        capacity = allowed.size

    if nuniq > capacity:
        raise ValueError(f"Too many notes: {nuniq} unique IDs, but only {capacity} pitches available "
                         f"for scale '{scale}' in MIDI 0–127. Choose a larger scale or reduce uniqueness.")

    targets = allowed[start_idx:start_idx + nuniq]
    mapping = dict(zip(uniq, targets))
    return vals.map(mapping).fillna(60).astype(int).to_numpy()

# ------------- Render callback -------------
def render_midi(_btn=None):
    with log:
        log.clear_output()
        try:
            name, content = _get_uploaded_name_and_bytes(upload)
            if not content:
                print("No file uploaded. Choose a .csv or .npy above.")
                return

            # Load fixed-schema data
            if str(name).lower().endswith('.csv'):
                df = pd.read_csv(io.BytesIO(content), header=None)
            elif str(name).lower().endswith('.npy'):
                arr = np.load(io.BytesIO(content), allow_pickle=True)
                df = pd.DataFrame(arr)
            else:
                print("Unsupported file type. Use .csv or .npy.")
                return

            if df.shape[1] < 6:
                print(f"Need ≥6 columns (track,channel,note,velocity,start_s,end_s). Got {df.shape[1]}.")
                return

            # Apply strict unique mapping if a scale is chosen (not chromatic)
            scale_name = scale_dd.value or "Chromatic (12-tone)"
            if not str(scale_name).startswith("Chromatic"):
                df[2] = map_unique_to_scale_ladder_strict(df[2], root_dd.value, scale_name)
                print(f"Applied unique ladder: {len(np.unique(df[2]))} unique pitches in '{root_dd.value} {scale_name}'")

            # Prepare MIDI
            bpm = bpm_slider.value
            tpb = tpb_slider.value
            tempo = mido.bpm2tempo(bpm)
            mid = mido.MidiFile(type=1, ticks_per_beat=tpb)

            # Track 0: tempo/meta
            meta = mido.MidiTrack()
            meta.append(MetaMessage('set_tempo', tempo=tempo, time=0))
            mid.tracks.append(meta)

            def sec2ticks(s): return int(round(s * tpb * (1_000_000 / tempo)))

            # Build tracks by column 0
            for trk_id in sorted(df[0].unique()):
                td = df[df[0] == trk_id].copy()
                td[4] = pd.to_numeric(td[4], errors='coerce')
                td[5] = pd.to_numeric(td[5], errors='coerce')
                td = td.dropna(subset=[4, 5]).sort_values(4)

                events = []
                for _, r in td.iterrows():
                    ch, note, vel = int(r[1]), int(r[2]), int(np.clip(r[3],1,127))
                    t_on, t_off = float(r[4]), float(r[5])
                    if t_off <= t_on: t_off = t_on + 1e-4
                    events += [(sec2ticks(t_on), 1, note, vel, ch),
                               (sec2ticks(t_off), 0, note, 0,   ch)]
                events.sort(key=lambda x: (x[0], x[1]))  # note-off before on if same tick

                tr = mido.MidiTrack(); mid.tracks.append(tr)
                prev = 0
                for tick, onoff, note, vel, ch in events:
                    dt = max(0, tick - prev)
                    tr.append(mido.Message('note_on' if onoff else 'note_off',
                                           note=int(note), velocity=int(vel),
                                           time=int(dt), channel=int(ch)))
                    prev = tick

            out_name = (str(name).rsplit('.',1)[0] or 'visond') + '_ViSoND.mid'
            mid.save(out_name)
            print(f"Saved {out_name}")
            display(FileLink(out_name))
            display(HTML(f'<a href="/files/{out_name}" download>Download {out_name}</a>'))
        except ValueError as e:
            # Show clean error (e.g., "Too many notes ...")
            print(str(e))
        except Exception as e:
            # Fallback: show message + type
            print(f"Error: {type(e).__name__}: {e}")

render_btn.on_click(render_midi)
print("UI ready: upload → set options → Render MIDI")

UI ready: upload → set options → Render MIDI
